# Forth Problem

## Imports

In [7]:
import sys
sys.path.append("../../")
sys.path.append("../")
from dlpmln import DeepLPMLN
import torch

import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

from dataGen import create_data_sample,format_dataList,format_observations,add_test,carry_test

## dprogram

Specify the DeepLPMLN program.

In [8]:
dprogram='''
num1(1,num1_1).
num1(0,num1_0).
num2(1,num2_1).
num2(0,num2_0).
1{carry(0,Carry): Carry=0..1}1.
nn(m1(A, B, Carry, 1), result, [0,1,2,3,4,5,6,7,8,9]) :- num1(P,A), num2(P,B), Carry=0..1.
nn(m2(A, B, Carry, 1), carry, [0,1]) :- num1(P,A), num2(P,B), Carry=0..1.

result(P,X) :- num1(P, A), num2(P, B), carry(P, Carry), result(A,B,Carry,0,X).
carry(P+1,X) :- num1(P, A), num2(P, B), carry(P, Carry), carry(A,B,Carry,0,X).
'''

## Neural Network Class

Create the neural network class that will be used for our data.

In [9]:
class FC(nn.Module):

    def __init__(self, *sizes):
        super(FC, self).__init__()
        layers = []
        for i in range(len(sizes)-2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-2], sizes[-1]))
        layers.append(nn.Softmax(1))
        self.nn = nn.Sequential(*layers)

    def forward(self, x):
        return self.nn(x)

## Neural Network Instantiation
- Instantiate neural networks.
- Map neural networks into the NNMapping function called 'function'.
- Specify the optimizers for each network (we use the Adam optimizer here).

In [10]:
m1 = FC(30,25,10) # network for adding the numbers
m2 = FC(30,5,2)   # network for finding the carry out


functions = {'m1':m1, 'm2':m2} #NNMapping

optimizers = {'m1':torch.optim.Adam(m1.parameters(), lr=0.01),'m2':torch.optim.Adam(m2.parameters(), lr=0.01)}


Below are 20 samples of data.

In [11]:
for i in range(20):
    print(create_data_sample()[0])

add([4,8],[7,9],1,[1,2,8]).
add([4,5],[6,8],1,[1,1,4]).
add([1,1],[0,4],0,[0,1,5]).
add([5,2],[8,8],1,[1,4,1]).
add([6,7],[1,3],0,[0,8,0]).
add([7,8],[3,0],0,[1,0,8]).
add([9,5],[3,0],1,[1,2,6]).
add([0,6],[3,3],1,[0,4,0]).
add([5,6],[3,4],0,[0,9,0]).
add([9,0],[6,9],0,[1,5,9]).
add([1,8],[1,0],1,[0,2,9]).
add([2,1],[5,8],1,[0,8,0]).
add([1,2],[5,3],0,[0,6,5]).
add([7,0],[7,3],0,[1,4,3]).
add([4,2],[3,1],0,[0,7,3]).
add([6,2],[4,6],1,[1,0,9]).
add([9,8],[2,3],0,[1,2,1]).
add([0,0],[0,3],1,[0,0,4]).
add([2,9],[6,4],1,[0,9,4]).
add([6,2],[2,3],0,[0,8,5]).


In [12]:
print('\n\n')

obs,str_list=create_data_sample() #data sample the same as DeepProbLog used 
print(obs+'\n\n')

dl=format_dataList(obs,str_list) # dataList item
print(dl)

o=format_observations(obs,str_list) # obsList item
print(o)




add([2,1],[0,5],0,[0,2,6]).


{'num1_0,num2_0,0': tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_0,num2_0,1': tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,0': tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,1': tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])}

    :-not carry(0,0).
    :-not carry(2,0).
    :-not result(1,2).
    :-not result(0,6).
    


## Create dataList and obsList 

This the code that will use format_dataList() and format_observations() to build the data set that our DeepLPMLN object will train on.

In [13]:
dataList = []
obsList = []

train_size=512
test_size=512


for i in range(train_size):
    
    obs,str_list=create_data_sample()
    
    dataList.append(format_dataList(obs,str_list))
    obsList.append(format_observations(obs,str_list))

## Create test dataset and dataLoader for neural network "m1"

In [14]:
add_test_dataset=add_test(test_size)

add_test_dataloader=DataLoader(add_test_dataset,batch_size=4,shuffle=True)

carry_test_dataset=carry_test(test_size)

carry_test_dataloader=DataLoader(carry_test_dataset,batch_size=4,shuffle=True)



## Create DeepLPMLN object

In [15]:
dlpmlnObj = DeepLPMLN(dprogram, functions, optimizers, dynamicMVPP=False)
#dlpmlnObj.device='cpu' #put the training on the CPU

## Training and Testing 

In [ ]:
print('training...')

for i in range(40):
    time1 = time.time()
    dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
    time2 = time.time()
    dlpmlnObj.testNN("m1", add_test_dataloader) #test m1 network
    dlpmlnObj.testNN("m2", carry_test_dataloader) #test m2 network
    print("--- train time: %s seconds ---" % (time2 - time1))
    print("--- test time: %s seconds ---" % (time.time() - time2))

training...
Training for epoch 1 ...
